In [8]:
%matplotlib qt

import numpy as np
import matplotlib.pyplot as plt
from math import gcd
from fractions import Fraction

def plot_phasor_diagram(Q, p, m):
    """
    Generates a phasor diagram for an electric motor and returns an array of phasor details.
    
    Parameters:
    - Q: Total number of slots in the motor.
    - p: Total number of poles in the motor.
    - m: Number of phases in the motor (e.g., 3 for a three-phase motor).
    
    Returns:
    - A list of [phasor_number, normalized_angle, phase, layer] quadruples, where:
      - phasor_number: Integer from 1 to Q, identifying each phasor.
      - normalized_angle: Angle in degrees [0, 360), representing the phasor's position.
      - phase: Phase designation ("phase -U", "phase +W", etc.).
      - layer: Layer index (0 to t-1), where t is the number of layers.
    The array is sorted first by layer (0, 1, ..., t-1), then by normalized_angle within each layer.
    The plot includes phasor numbers and phase designations (e.g., "-U") matching the sorted array, with reduced text size.
    """
    # --- Calculate derived parameters ---
    # PP: Number of pole-pairs (half the number of poles)
    PP = int(p / 2)
    # t: Greatest common divisor between Q and PP, determines the number of layers
    t = gcd(Q, PP)
    # Qp: Number of phasors per layer (Q divided by t)
    Qp = Q / t
    # phasors_per_layer: Integer number of phasors in each layer
    phasors_per_layer = int(Qp)

    # --- Calculate slot per pole per phase ---
    # q: Fraction representing slots per pole per phase (Q / (p * m))
    q = Fraction(Q, p * m)
    z = q.numerator  # Numerator of the fraction
    n = q.denominator  # Denominator of the fraction, used to determine winding type

    # --- Calculate angular displacements ---
    # alpha_u: Angular displacement between consecutive phasors in degrees
    # alpha_z: Base angle for phasor spacing
    if t == PP:
        # Case 1: When t equals PP, use standard angle calculation
        alpha_u = 360 * PP / Q
        alpha_z = alpha_u
    else:
        # Case 2: For fractional slot designs, adjust angles based on Pyrhonen's method
        alpha_z = 360 / Qp
        if n % 2 != 0:  # If n is odd (first-grade winding)
            alpha_u = n * alpha_z
        else:  # If n is even
            alpha_u = n / 2 * alpha_z

    # --- Set up the plot ---
    # Create a figure and axis for the phasor diagram (5x5 inches, 150 DPI)
    fig, ax = plt.subplots(figsize=(5, 5), dpi=150)
    ax.set_aspect('equal')  # Ensure the plot is circular (equal scaling on axes)
    # Add a light gray grid for better visualization
    ax.grid(True, color='lightgray', linestyle='--', linewidth=0.5, alpha=0.5)

    # --- Define layer radii ---
    # Each layer has a radius starting at 1, increasing by 0.5 per layer
    # For t=1, radii = [1]; for t=2, radii = [1, 1.5], etc.
    radii = [1 + 0.5 * i for i in range(t)]

    # --- Initialize plotting parameters ---
    # Start the first phasor at 90 degrees (positive y-axis, top of the plot)
    current_angle = 90
    # Define the length of the arrowhead (in plot units, kept small)
    head_length = 0.05
    # List of phasor numbers to be colored red in the plot (example values)
    U_phasors = []

    # --- Initialize array to store phasor data ---
    # phasor_angles: Stores [phasor_number, normalized_angle, layer] temporarily
    phasor_angles = []

    # --- Collect phasor data during plotting ---
    phasor_number = 1  # Start numbering phasors from 1
    for layer in range(t):
        # Get the radius for the current layer (e.g., 1 for layer 0, 1.5 for layer 1)
        radius = radii[layer]
        # Adjust shaft length so the arrow tip (including head) touches the circle
        shaft_scale = 1 - (head_length / radius)
        
        for i in range(phasors_per_layer):
            # Normalize the current angle to [0, 360) to avoid negative/large angles
            normalized_angle = current_angle % 360
            # Store phasor data: number, angle, layer
            phasor_angles.append([phasor_number, normalized_angle, layer])
            
            # --- Plot the phasor ---
            # Convert angle to radians for trigonometric calculations
            angle_rad = np.deg2rad(current_angle)
            # Calculate x, y coordinates of the arrow tip, scaled to account for arrowhead
            x = radius * shaft_scale * np.cos(angle_rad)
            y = radius * shaft_scale * np.sin(angle_rad)
            # Color the phasor red if its number is in U_phasors, else black
            color = 'red' if phasor_number in U_phasors else 'black'
            # Draw an arrow from origin (0,0) to (x,y) with specified head size
            ax.arrow(0, 0, x, y, head_width=0.04, head_length=0.05, fc=color, ec=color)
            
            # --- Placeholder for text label (added after phase assignment) ---
            # Store text position for later use
            text_offset = 1.08 * radius
            text_angle_offset = 7
            text_x = text_offset * np.cos(angle_rad - text_angle_offset / 180 * np.pi)
            text_y = text_offset * np.sin(angle_rad - text_angle_offset / 180 * np.pi)
            phasor_angles[-1].append((text_x, text_y))  # Append text coordinates
            
            # Increment phasor number
            phasor_number += 1
            # Move to the next phasor angle (clockwise, subtract alpha_u)
            current_angle -= alpha_u

    # --- Sort phasors by layer and angle ---
    # Create a list of t empty lists to group phasors by layer
    layered_phasors = [[] for _ in range(t)]
    # Assign each phasor to its layer's list
    for phasor in phasor_angles:
        layered_phasors[phasor[2]].append([phasor[0], phasor[1], phasor[2], phasor[3]])
    
    # Sort each layer's phasors by normalized_angle, starting at 90 degrees
    # The key (x[1] - 90) % 360 transforms angles so 90° maps to 0, 91° to 1, ..., 89° to 359
    for layer_phasors in layered_phasors:
        layer_phasors.sort(key=lambda x: (x[1] - 90) % 360)
    
    # Concatenate sorted layers into a single list
    # This ensures layer 0 phasors come first, then layer 1, etc.
    sorted_phasors = []
    for layer_phasors in layered_phasors:
        sorted_phasors.extend(layer_phasors)

    # --- Assign phase designations ---
    # Each phase group has Q/t/m/2 phasors
    elements_per_phase = int(Q / t / m / 2)
    # Define the phase order to cycle through
    phase_order = ["phase -U", "phase +W", "phase -V", "phase +U", "phase -W", "phase +V"]
    # Generate phase assignments for all Q phasors
    total_phasors = Q
    phase_assignments = []
    while len(phase_assignments) < total_phasors:
        for phase in phase_order:
            phase_assignments.extend([phase] * elements_per_phase)
            if len(phase_assignments) >= total_phasors:
                break
    phase_assignments = phase_assignments[:total_phasors]

    # --- Create mapping from phasor_number to phase ---
    # Map phasor numbers to their phases in the sorted array
    phasor_to_phase = {p[0]: phase_assignments[i] for i, p in enumerate(sorted_phasors)}

    # --- Map full phase names to short forms for plot labels ---
    phase_short_map = {
        "phase -U": "-U", "phase +W": "+W", "phase -V": "-V",
        "phase +U": "+U", "phase -W": "-W", "phase +V": "+V"
    }

    # --- Add text labels to the plot ---
    # Iterate through phasor_angles to add labels using sorted phases
    for phasor in phasor_angles:
        phasor_number = phasor[0]
        text_x, text_y = phasor[3]  # Text coordinates stored during plotting
        # Get the phase from the sorted array's mapping
        phase = phasor_to_phase[phasor_number]
        # Combine phasor number and shortened phase (e.g., "1 (-U)")
        label = f"{phasor_number} ({phase_short_map[phase]})"
        # Add the text with reduced font size
        ax.text(text_x, text_y, label, fontsize=6, ha='center', va='center')

    # --- Create final array with phase designations ---
    # Format: [phasor_number, normalized_angle, phase, layer]
    phasor_angles_with_phase = [[p[0], p[1], phase_assignments[i], p[2]] for i, p in enumerate(sorted_phasors)]

    # --- Draw layer circles (if t > 1) ---
    # Add circles to visualize layers, only if there are multiple layers
    if t > 1:
        for radius in radii:
            circle = plt.Circle((0, 0), radius, fill=False, color='gray')
            ax.add_patch(circle)

    # --- Set plot limits ---
    # Ensure all phasors and circles are visible by setting limits based on the largest radius
    max_radius = max(radii)
    ax.set_xlim(-max_radius - 0.5, max_radius + 0.5)
    ax.set_ylim(-max_radius - 0.5, max_radius + 0.5)

    # --- Add plot labels and title ---
    ax.set_xlabel('Real')  # X-axis represents the real part of the phasor
    ax.set_ylabel('Imaginary')  # Y-axis represents the imaginary part
    ax.set_title('Phasor Diagram')  # Title of the plot

    # --- Save and close the plot ---
    plt.savefig('phasor_diagram.png')  # Save the plot as an image
    plt.show()
    # plt.close(fig)  # Close the figure to free memory

    # --- Return the final array ---
    return phasor_angles_with_phase

def generate_phasor_list_for_X_phase(diagram_output, phase):
    """
    Generates a phasor_list from plot_phasor_diagram output for the specified phase
    ("U", "V", or "W"), including both positive and negative phasors (e.g., "+U" and "-U").
    
    Parameters:
    - diagram_output: List of [phasor_number, normalized_angle, phase, layer] quadruples
                      from plot_phasor_diagram.
    - phase: String specifying the target phase ("U", "V", or "W").
    
    Returns:
    - phasor_list: List of (phasor_number, magnitude, angle) tuples, where:
      - phasor_number: Integer identifying the phasor (1 to Q).
      - magnitude: 1 for positive phase (e.g., "phase +U"), -1 for negative (e.g., "phase -U").
      - angle: Normalized angle from diagram_output (degrees, [0, 360)).
      - Length equals the number of phasors for the specified phase.
    
    Raises:
    - ValueError: If phase is not "U", "V", or "W".
    
    Note:
    For use with plot_multiple_phasor_sum, extract (magnitude, angle) from each tuple,
    as it expects a list of (phasor_number, magnitude, angle) tuples but uses only
    magnitude and angle for plotting.
    """
    # Validate phase parameter
    if phase not in ["U", "V", "W"]:
        raise ValueError("Phase must be 'U', 'V', or 'W'")
    
    # Initialize phasor_list
    phasor_list = []
    
    # Define positive and negative phase names
    positive_phase = f"phase +{phase}"
    negative_phase = f"phase -{phase}"
    
    # Iterate through diagram_output to find phasors for the specified phase
    for phasor in diagram_output:
        phasor_number, normalized_angle, phase_name, layer = phasor
        if phase_name in [positive_phase, negative_phase]:
            # Assign magnitude based on phase
            magnitude = -1 if phase_name == negative_phase else 1
            # Add tuple to phasor_list with phasor_number
            phasor_list.append((phasor_number, magnitude, normalized_angle))
    
    return phasor_list


def plot_multiple_phasor_sum(phasors, input_thickness=0.005, resultant_thickness=0.007, ax=None, resultant_color="green", resultant_label="Geometric sum"):
    """
    Plot multiple phasors and their successive sums with customizable thickness and resultant color.
    
    Parameters:
    phasors (list): List of tuples, each containing (phasor_number, magnitude, angle) where:
                    - phasor_number: Integer identifying the phasor.
                    - magnitude: Float, magnitude of the phasor.
                    - angle: Angle in degrees.
    input_thickness (float): Thickness of input phasor arrows (default: 0.005).
    resultant_thickness (float): Thickness of resultant phasor arrow (default: 0.007).
    ax (matplotlib.axes.Axes, optional): Axis to plot on; if None, creates a new figure.
    resultant_color (str): Color of the resultant vector (default: "green").
    resultant_label (str): Label for the resultant vector in the legend (default: "Geometric sum").

    Example phasors:
    phasors = [
        (1, -1.0, 90),   # Phasor 1: -U
        (8, -1.0, 120),  # Phasor 8: -U
        (7, 1.0, 270),   # Phasor 7: +U
        (2, 1.0, 300)    # Phasor 2: +U
    ]

    Notes:
    Uses quiver() to plot arrows representing:
       Input phasors (alternating black/grey, labeled with phasor_number).
       Resultant phasor (custom color, e.g., blue for U phase).
    The "angles='xy'" parameter ensures arrows point correctly based on calculated components.
    """
    # Create new figure if no axis is provided
    if ax is None:
        fig, ax = plt.figure(figsize=(8, 6)), plt.gca()
    
    # Initialize lists to store components
    x_components = []
    y_components = []
    
    # Calculate components for each phasor
    for phasor_number, magnitude, angle in phasors:
        theta = np.radians(angle)
        x = magnitude * np.cos(theta)
        y = magnitude * np.sin(theta)
        x_components.append(x)
        y_components.append(y)

    # Starting point
    current_x = 0
    current_y = 0
    
    # Plot each phasor
    cycle_colors = ['black', 'grey']
    num_colors = len(cycle_colors)
    for i, (phasor_number, magnitude, angle) in enumerate(phasors):
        color_to_use = cycle_colors[i % num_colors]
        x = x_components[i]
        y = y_components[i]
        ax.quiver(current_x, current_y, x, y, angles='xy', scale_units='xy', scale=1, 
                  color=color_to_use, width=input_thickness,
                  label=f'Phasor {phasor_number}: {magnitude}∠{angle}°')
        current_x += x
        current_y += y
    
    # Plot resultant
    resultant_x = sum(x_components)
    resultant_y = sum(y_components)
    magnitude_resultant = np.sqrt(resultant_x**2 + resultant_y**2)
    angle_resultant = np.degrees(np.arctan2(resultant_y, resultant_x))
    ax.quiver(0, 0, resultant_x, resultant_y, angles='xy', scale_units='xy', scale=1, 
              color=resultant_color, width=resultant_thickness,
              label=f'{resultant_label}: {magnitude_resultant:.2f}∠{angle_resultant:.1f}°')

    # Set plot parameters
    ax.set_aspect('equal')
    ax.grid(True)
    ax.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    ax.axvline(x=0, color='k', linestyle='-', alpha=0.3)

    # Set limits
    max_range = max(magnitude_resultant, *[abs(mag) for _, mag, _ in phasors]) * 2
    ax.set_xlim(-max_range, max_range)
    ax.set_ylim(-max_range, max_range)

    # Add labels
    ax.set_xlabel('Real')
    ax.set_ylabel('Imaginary')


# Generate diagram output
diagram_output = plot_phasor_diagram(Q=24, p=28, m=3)

# Define phases for m=3
phases = ["U", "V", "W"]
phases_color = ["green", "red", "blue"]

# Create a figure with three subplots (3 rows, 1 column)
fig, ax = plt.subplots(figsize=(8, 12), sharex=True, sharey=True)

# Generate and plot phasor sums for each phase
for phase, phase_color in zip(phases, phases_color):
    # Generate phasor_list for the current phase
    phasor_list = generate_phasor_list_for_X_phase(diagram_output, phase)
    # Plot phasor sum on the specified axis
    plot_multiple_phasor_sum(phasor_list, input_thickness=0.005, resultant_thickness=0.007, ax=ax, resultant_color = phase_color, resultant_label = "Phase " + phase)
    # # Set subplot title
    ax.set_title(f'Phasor Sum')
    # # Add legend
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Adjust layout to prevent overlap
plt.tight_layout()

# Add overall figure title
fig.suptitle('Phasor Sums for All Phases', y=1.02)

# Show the plot
plt.show()

